In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
path = 'CSV 사출데이터/'
file_list = os.listdir(path)
file_df = pd.DataFrame(file_list)
black_list = [4,9,14,18,23,30,35]
file_list = file_df[file_df.index.isin(black_list)== False][0].to_list()

In [ ]:
for slide in tqdm(range(0,6)):
    total_df = pd.DataFrame()
    group_number = 0
    for file in tqdm(file_list):
        try : df = pd.read_csv(path+ file)
        except: 
            df = pd.read_csv(path+file,encoding='CP949')
            df.rename({df.columns[0]:'일시'},axis=1)   
        df = df[(df.INM005_H_05_09_464 >=3)&(df.INM005_H_05_09_464 <5.8)].reset_index(drop=True)
        df[['투입수량','양품수량','불량수량']] = df[['투입수량','양품수량','불량수량']].shift(slide)
        df = df.iloc[slide:].reset_index(drop=True)
        start = 1
        integer = 0
        decimal = 0
        result = []
        result_group_number = []
        for n, i in enumerate(range(len(df))):
            if start == 1:
                integer, decimal_point = str(df.iloc[i].INM005_H_05_09_464).split('.')
                integer, decimal_point = int(integer), float(decimal_point)
                if decimal >= 1:
                    decimal -= 1
                    integer += 1
                if n == 0:
                    result_group_number.append(group_number)
                    result.append(df.iloc[i])
                    start += 1
                    decimal = decimal+ decimal_point*0.1
                else:
                    result_group_number.append(group_number)
                    group_number+=1
                    start += 1
                    result_group_number.append(group_number)
                    result.append(df.iloc[i])
                    result.append(df.iloc[i])
                    decimal = decimal+ decimal_point*0.1
            else:
                result_group_number.append(group_number)
                result.append(df.iloc[i])
                start += 1
                if start < integer: continue
                else: start -= integer
        df1 = pd.DataFrame(result, columns = df.columns)
        df1['group'] = result_group_number
        df1['투입수량'] = (df1['투입수량'] - df1['투입수량'].shift()).fillna(0)
        df1['양품수량'] = (df1['양품수량'] - df1['양품수량'].shift()).fillna(0)
        df1['불량수량'] = (df1['불량수량'] - df1['불량수량'].shift()).fillna(0)
        df1 = df1[df1.group != group_number]
        total_df = pd.concat([total_df,df1])
    total_df.to_csv(str(slide)+'_total_df.csv')

In [ ]:
import pandas as pd
for slide in range(6):
    total_df = pd.read_csv(str(slide)+'_total_df.csv')
    total_df = total_df.drop(['일시','Unnamed: 0','?쇱떆'],axis=1 )
    drop_cols = [col for col in total_df.columns if total_df[col].nunique() ==1]
    total_df = total_df.drop(drop_cols,axis=1)
    nunique_cols = total_df.drop(['투입수량','양품수량','불량수량'],axis=1).groupby('group').nunique()
    nunique_cols = [col for col in nunique_cols.columns if nunique_cols[col].nunique() == 1]
    total_df = total_df.drop(nunique_cols, axis=1)
    label_cols = ['투입수량','양품수량','불량수량']
    label_df  = total_df[label_cols+['group']]
    label_df= label_df.groupby('group').sum().reset_index()
    unique_cols = total_df.drop(label_cols,axis=1)
    unique_cols.groupby('group').nunique()['INM005_H_05_09_053'].value_counts()
    unique_cols = unique_cols.groupby('group').nunique()
    unique_cols = [col for col in unique_cols.columns if unique_cols[col].nunique() <2]
    if len(unique_cols) == 0:
        pass
    else:
        total_df = total_df.drop(unique_cols,axis=1)
        unique_df = pd.merge(total_df[unique_cols + ['group']]).groupby('group').mean().reset_index()
        label_df = pd.merge(label_df, unique_df, on='group') 
    min_df = total_df.drop(label_cols,axis=1).groupby('group').max().reset_index()
    max_df = total_df.drop(label_cols,axis=1).groupby('group').max().reset_index()
    final_df = pd.merge(label_df, min_df, on='group')
    final_df = pd.merge(final_df, max_df, on='group')
    final_df = final_df[final_df.투입수량 == 8].reset_index(drop=True)
    final_df = final_df.drop('group',axis=1)
    drop_cols = [col for col in final_df.columns if final_df[col].nunique() ==1]
    final_df = final_df.drop(drop_cols, axis=1)
    final_df['label'] = final_df.불량수량.map(lambda x: 1 if x != 0 else 0)
    final_df = final_df.drop(['양품수량','불량수량'],axis=1)
    final_df.to_csv(str(slide)+'_final_df.csv',index=False)

In [6]:
from sklearn.metrics import mean_squared_error
import math
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import mean_absolute_error, r2_score
from optuna.samplers import TPESampler
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from interpret.glassbox import ExplainableBoostingRegressor, ExplainableBoostingClassifier
from interpret import show
import optuna
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import f1_score, accuracy_score
import joblib
import numpy as np

class EarlyStoppingCallback(object):
    """Early stopping callback for Optuna."""
    def __init__(self, early_stopping_rounds: int, direction: str = "minimize") -> None:
        self.early_stopping_rounds = early_stopping_rounds
        self._iter = 0
        if direction == "minimize":
            self._operator = operator.lt
            self._score = np.inf
        elif direction == "maximize":
            self._operator = operator.gt
            self._score = -np.inf
        else:
            ValueError(f"invalid direction: {direction}")
    def __call__(self, study: optuna.Study, trial: optuna.Trial) -> None:
        """Do early stopping."""
        if self._operator(study.best_value, self._score):
            self._iter = 0
            self._score = study.best_value
        else:
            self._iter += 1
        if self._iter >= self.early_stopping_rounds:
            study.stop()

def hist_create(trial):
    param = {
        'max_iter' : trial.suggest_int('max_iter',25,50),
        'l2_regularization' : trial.suggest_float('l2_regularization', 0.1,2.5,step = 0.0000005),
        'learning_rate' : trial.suggest_float('learning_rate', 0.1,2.5,step = 0.0005),
        'max_depth' : trial.suggest_int('max_depth', 10, 20),
        'max_bins' : trial.suggest_int('max_bins', 10, 255),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 10, 20),
        'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 20, 35),
        'random_state': trial.suggest_categorical('random_state', [2023]),
    }
    def get_model(param, REG):
        if REG == True:model = HistGradientBoostingRegressor(**param)
        else: model = HistGradientBoostingClassifier(**param)
        return model
    model = get_model(param, REG)
    return model

def ebm_create(trial):
    param = {
        'learning_rate' : trial.suggest_float('learning_rate', 0.1,2.5,step = 0.0005),
        'max_leaves' : trial.suggest_int('max_leaves',3,20),
        'random_state': trial.suggest_categorical('random_state', [2023])
    }
    def get_model(param, REG):
        if REG == True:model = ExplainableBoostingRegressor(**param)
        else: model = ExplainableBoostingClassifier(**param)
        return model
    model = get_model(param, REG)
    return model

def rf_create(trial):
    param = {
        'max_depth' : trial.suggest_int('max_depth', 5, 20),
        'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 2, 1000),
        'n_estimators' :  trial.suggest_int('n_estimators', 100, 1000),
        'random_state': trial.suggest_categorical('random_state', [2023])
    }
    def get_model(param, REG):
        if REG == True:model = RandomForestRegressor(**param)
        else: model = RandomForestClassifier(**param)
        return model
    model = get_model(param, REG)
    return model

def ada_create(trial):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators',10,80,step=2),
        'learning_rate' :  trial.suggest_float('learning_rate', 0.1,2.5,step = 0.0000005),
        'random_state': trial.suggest_categorical('random_state', [2023])
    }
    def get_model(param, REG):
        if REG == True:model = AdaBoostRegressor(**param)
        else: model = AdaBoostClassifier(**param)
        return model
    model = get_model(param, REG)

    return model

def cat_create(trial):
    param = {
#         "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        'iterations' : trial.suggest_int('iterations', 5, 30),
        'od_wait':trial.suggest_int('od_wait', 100, 500),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
#         'subsample': trial.suggest_uniform('subsample',0,1),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,10),
        'depth' : trial.suggest_int('depth', 3, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),  
        'task_type':"CPU",
#         'colsample_bylevel' :  trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        'random_state': trial.suggest_categorical('random_state', [2023]),

        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"])
    }


    def get_model(param, REG):
        if REG == True:model = CatBoostRegressor(**param)
        else: model = CatBoostClassifier(**param)
        return model
    model = get_model(param, REG)    
    return model


    def get_model(param, REG):
        if REG == True:model = CatBoostRegressor(**param)
        else: model = CatBoostClassifier(**param)
        return model
    model = get_model(param, REG)    
    return model

def ert_create(trial):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 50, 120),
        'max_depth' : trial.suggest_int('max_depth', 5, 16),
        'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 15, 25),
        'random_state': trial.suggest_categorical('random_state', [2023])
    }

    def get_model(param, REG):
        if REG == True:model = ExtraTreesRegressor(**param)
        else: model = ExtraTreesClassifier(**param)
        return model
    model = get_model(param, REG)

    return model


def xgb_create(trial):
    param = {
        'n_estimators' : trial.suggest_int("n_estimators", 20, 1000),
        'max_depth' : trial.suggest_int('max_depth', 3, 20),
        'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'min_child_weight' :  trial.suggest_int('min_child_weight', 1, 300),
        'tree_method':'gpu_hist',  #얘가 쓰는중
        'lambda': trial.suggest_loguniform('lambda', 1e-4, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-4, 10.0),
        'random_state': trial.suggest_categorical('random_state', [2023])}
    def get_model(param, REG):
        if REG == True:model = XGBRegressor(**param)
        else: model = XGBClassifier(**param)
        return model
    model = get_model(param, REG)
    return model

def tabnet_create(trial):
    param = {
        'mask_type' : trial.suggest_categorical("mask_type", ["entmax", "sparsemax"]),
        'n_steps' : trial.suggest_int("n_steps", 1, 3, step=1),
        'gamma' : trial.suggest_float("gamma", 1.0, 2.0, step=0.2),
        'n_independent' : trial.suggest_int("n_independent", 1, 5),
        'n_shared' : trial.suggest_int("n_shared", 1, 5),
        'lambda_sparse' : trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)}
    
    def get_model(param, REG):
        if REG == True:model = TabNetRegressor(**param)
        else: model = TabNetClassifier(**param)
        return model
    
    model = get_model(param, REG)

    return model

def lgbm_create(trial):
    param = {
        'num_leaves': trial.suggest_int("num_leaves", 100, 500),
        'n_estimators' : trial.suggest_int("n_estimators", 5, 1000),
        'max_depth' : trial.suggest_int('max_depth', 3,20),
        'min_child_samples' : trial.suggest_int('min_child_samples', 100, 1200),
        'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 1, 1000),
        'random_state': trial.suggest_categorical('random_state', [2023])
    }

    def get_model(param, REG):
        if REG == True:model = LGBMRegressor(**param)
        else: model = LGBMClassifier(**param)
        return model
    model = get_model(param, REG)
    return model

def objective(trial):
    if model_name == 'xgb' :  model = xgb_create(trial) #2018
    elif model_name == 'lgbm':  model = lgbm_create(trial) #2018
    elif model_name == 'rf' : model = rf_create(trial)
    elif model_name == 'ert' : model = ert_create(trial)
    elif model_name == 'ada' : model = ada_create(trial) # 2009
    elif model_name == 'cat' : model = cat_create(trial)
    elif model_name == 'hist' : model = hist_create(trial) #2019
    elif model_name == 'ebm' : model = ebm_create(trial) #2022
    elif model_name == 'tab' : model = tabnet_create(trial) #2022
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    if REG == False: score = f1_score(preds, y_valid)
    else: score = mean_absolute_error(preds, y_valid)
    return score

def get_model(model_name, best_params):
    if REG == False:
        if model_name == 'xgb' :  model = XGBClassifier(**best_params[model_name])
        elif model_name == 'lgbm':  model = LGBMClassifier(**best_params[model_name])
        elif model_name == 'rf' : model = RandomForestClassifier(**best_params[model_name])
        elif model_name == 'ert' : model = ExtraTreesClassifier(**best_params[model_name])
        elif model_name == 'ada' : model = AdaBoostClassifier(**best_params[model_name])
        elif model_name == 'cat' : model = CatBoostClassifier(**best_params[model_name])
        elif model_name == 'hist' : model = HistGradientBoostingClassifier(**best_params[model_name])
        elif model_name == 'ebm' : model = ExplainableBoostingClassifier(**best_params[model_name])
    else:
        if model_name == 'xgb' :  model = XGBRegressor(**best_params[model_name])
        elif model_name == 'lgbm':  model = LGBMRegressor(**best_params[model_name])
        elif model_name == 'rf' : model = RandomForestRegressor(**best_params[model_name])
        elif model_name == 'ert' : model = ExtraTreesRegressor(**best_params[model_name])
        elif model_name == 'ada' : model = AdaBoostRegressor(**best_params[model_name])
        elif model_name == 'cat' : model = CatBoostRegressor(**best_params[model_name])
        elif model_name == 'hist' : model = HistGradientBoostingRegressor(**best_params[model_name])
        elif model_name == 'ebm' : model = ExplainableBoostingRegressor(**best_params[model_name])
        elif model_name == 'tab' : model = TabNetRegressor(**best_params[model_name])
    return model

def get_corr(df_anova,i):
    lists = []
    x = df_anova.corr()
    x = x.reset_index()
    col_list = x['index']
    x.drop(['index'], axis=1, inplace=True)
    columns = x.columns.drop(target)
    for col in columns:
        indexes = x[col][(abs(x[col]) > i) & (x[col] < 1)].index
        if len(indexes) != 0:
            max_index = x[x[target] == np.max(x[target][indexes])].index
            indexes = indexes.drop(max_index)
        for i in indexes:
            lists.append(i)
    final_list = []
    for i in lists:
        if i not in final_list:
            final_list.append(i)
    final_columns = x.drop(col_list[final_list], axis=1).columns
    df_anova_corr = df_anova[final_columns].copy()
    return df_anova_corr
import operator


def get_train(model_name):
    sampler = TPESampler(seed=1000)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    early_stopping = EarlyStoppingCallback(200, direction='maximize')
    if( model_name == 'cat')|(model_name =='tab'):
        early_stopping = EarlyStoppingCallback(50, direction='maximize')
        study.optimize(objective, n_trials=200, callbacks = [early_stopping])#callbacks= [early_stopping]
    else : study.optimize(objective, n_trials=1200, callbacks= [early_stopping])
    best_params = {}
    best_params[model_name] = study.best_params

    score = study.best_value
    model = get_model(model_name, best_params)
    model.fit(X_train, y_train)
    joblib.dump(model, f'{model_name}_{i}.pkl')
    return model_name, model, best_params[model_name], score

In [7]:
i = 'fix'

In [13]:
import pandas as pd
total_list1 = []
import numpy as np
for slide in range(6):
    final_df = pd.read_csv(str(slide)+'_final_df.csv')
    final_df.loc[final_df.drop('label',axis=1).drop_duplicates().index].reset_index(drop=True)
    X_features = final_df.drop('label',axis=1)
    y_target = final_df.label
    X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size= 0.1 , shuffle = False)
    REG = False
    model_list = ['hist','rf','ebm','ada']
    for model_name in model_list:
        model_name, model, params, score = get_train(model_name)
        preds = model.predict(X_valid)
        total_list1.append([slide, model_name, params, score, accuracy_score(preds, y_valid)])

[I 2023-12-14 20:13:00,168] A new study created in memory with name: no-name-c3b6fb50-d384-4661-87b1-df2b10a362af
[I 2023-12-14 20:13:00,411] Trial 0 finished with value: 0.7465335551858014 and parameters: {'max_iter': 41, 'l2_regularization': 0.3760165, 'learning_rate': 2.3810000000000002, 'max_depth': 15, 'max_bins': 224, 'min_samples_leaf': 12, 'max_leaf_nodes': 20, 'random_state': 2023}. Best is trial 0 with value: 0.7465335551858014.
[I 2023-12-14 20:13:00,698] Trial 1 finished with value: 0.2763765541740675 and parameters: {'max_iter': 35, 'l2_regularization': 0.6595169999999999, 'learning_rate': 2.1205000000000003, 'max_depth': 12, 'max_bins': 192, 'min_samples_leaf': 14, 'max_leaf_nodes': 22, 'random_state': 2023}. Best is trial 0 with value: 0.7465335551858014.
[I 2023-12-14 20:13:00,946] Trial 2 finished with value: 0.40939384322437317 and parameters: {'max_iter': 44, 'l2_regularization': 0.266997, 'learning_rate': 2.225, 'max_depth': 20, 'max_bins': 239, 'min_samples_leaf': 

[I 2023-12-14 20:13:12,295] Trial 25 finished with value: 0.696522655426765 and parameters: {'max_iter': 43, 'l2_regularization': 0.7432924999999999, 'learning_rate': 1.0215, 'max_depth': 13, 'max_bins': 222, 'min_samples_leaf': 15, 'max_leaf_nodes': 22, 'random_state': 2023}. Best is trial 22 with value: 0.7752458623171026.
[I 2023-12-14 20:13:12,617] Trial 26 finished with value: 0.680986475735879 and parameters: {'max_iter': 47, 'l2_regularization': 0.139774, 'learning_rate': 1.971, 'max_depth': 15, 'max_bins': 142, 'min_samples_leaf': 13, 'max_leaf_nodes': 27, 'random_state': 2023}. Best is trial 22 with value: 0.7752458623171026.
[I 2023-12-14 20:13:13,072] Trial 27 finished with value: 0.5088547815820543 and parameters: {'max_iter': 38, 'l2_regularization': 0.4310345, 'learning_rate': 1.514, 'max_depth': 18, 'max_bins': 251, 'min_samples_leaf': 17, 'max_leaf_nodes': 29, 'random_state': 2023}. Best is trial 22 with value: 0.7752458623171026.
[I 2023-12-14 20:13:13,565] Trial 28 fi

[I 2023-12-14 20:13:23,510] Trial 51 finished with value: 0.4737327188940092 and parameters: {'max_iter': 39, 'l2_regularization': 0.6580809999999999, 'learning_rate': 0.5945, 'max_depth': 10, 'max_bins': 229, 'min_samples_leaf': 16, 'max_leaf_nodes': 20, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:24,010] Trial 52 finished with value: 0.5467922307239552 and parameters: {'max_iter': 41, 'l2_regularization': 0.8478239999999999, 'learning_rate': 0.767, 'max_depth': 10, 'max_bins': 246, 'min_samples_leaf': 17, 'max_leaf_nodes': 22, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:24,464] Trial 53 finished with value: 0.4042278937870585 and parameters: {'max_iter': 36, 'l2_regularization': 0.482611, 'learning_rate': 1.108, 'max_depth': 12, 'max_bins': 199, 'min_samples_leaf': 16, 'max_leaf_nodes': 21, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:25,069] 

[I 2023-12-14 20:13:37,581] Trial 77 finished with value: 0.3936904370686822 and parameters: {'max_iter': 47, 'l2_regularization': 0.428712, 'learning_rate': 0.754, 'max_depth': 13, 'max_bins': 160, 'min_samples_leaf': 16, 'max_leaf_nodes': 29, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:38,000] Trial 78 finished with value: 0.678648068669528 and parameters: {'max_iter': 45, 'l2_regularization': 0.9752044999999999, 'learning_rate': 1.191, 'max_depth': 15, 'max_bins': 227, 'min_samples_leaf': 11, 'max_leaf_nodes': 22, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:38,302] Trial 79 finished with value: 0.40086206896551724 and parameters: {'max_iter': 49, 'l2_regularization': 0.5298955, 'learning_rate': 2.3955, 'max_depth': 14, 'max_bins': 213, 'min_samples_leaf': 12, 'max_leaf_nodes': 20, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:38,765] Trial 80 

[I 2023-12-14 20:13:48,801] Trial 103 finished with value: 0.3461950696677385 and parameters: {'max_iter': 31, 'l2_regularization': 1.7402745, 'learning_rate': 2.1615, 'max_depth': 11, 'max_bins': 73, 'min_samples_leaf': 16, 'max_leaf_nodes': 29, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:49,127] Trial 104 finished with value: 0.7126349687440804 and parameters: {'max_iter': 34, 'l2_regularization': 1.4348565, 'learning_rate': 2.4365, 'max_depth': 10, 'max_bins': 124, 'min_samples_leaf': 15, 'max_leaf_nodes': 28, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:49,464] Trial 105 finished with value: 0.7432907643901536 and parameters: {'max_iter': 36, 'l2_regularization': 1.462217, 'learning_rate': 2.4305000000000003, 'max_depth': 10, 'max_bins': 106, 'min_samples_leaf': 15, 'max_leaf_nodes': 28, 'random_state': 2023}. Best is trial 50 with value: 0.8116504854368932.
[I 2023-12-14 20:13:49,860] Tr

...

[I 2023-12-15 00:03:42,417] Trial 25 finished with value: 0.6967467079783114 and parameters: {'n_estimators': 42, 'learning_rate': 2.2686945, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:03:43,300] Trial 26 finished with value: 0.809004329004329 and parameters: {'n_estimators': 22, 'learning_rate': 2.485176, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:03:43,865] Trial 27 finished with value: 0.809004329004329 and parameters: {'n_estimators': 14, 'learning_rate': 2.1830425, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:03:45,536] Trial 28 finished with value: 0.6295552367288378 and parameters: {'n_estimators': 42, 'learning_rate': 1.668961, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:03:47,669] Trial 29 finished with value: 0.19073359073359072 and parameters: {'n_estimators': 54, 'learning_rate': 2.3227155, 'random_state

[I 2023-12-15 00:04:25,292] Trial 64 finished with value: 0.7668955547524395 and parameters: {'n_estimators': 22, 'learning_rate': 2.0238989999999997, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:04:26,023] Trial 65 finished with value: 0.809004329004329 and parameters: {'n_estimators': 18, 'learning_rate': 2.2413275, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:04:27,061] Trial 66 finished with value: 0.809004329004329 and parameters: {'n_estimators': 26, 'learning_rate': 2.434638, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:04:28,029] Trial 67 finished with value: 0.809004329004329 and parameters: {'n_estimators': 24, 'learning_rate': 2.497052, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:04:28,678] Trial 68 finished with value: 0.809004329004329 and parameters: {'n_estimators': 16, 'learning_rate': 2.3337285, 'random

[I 2023-12-15 00:05:06,954] Trial 103 finished with value: 0.809004329004329 and parameters: {'n_estimators': 24, 'learning_rate': 2.175846, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:07,626] Trial 104 finished with value: 0.809004329004329 and parameters: {'n_estimators': 16, 'learning_rate': 2.3958805, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:08,837] Trial 105 finished with value: 0.4530531845042679 and parameters: {'n_estimators': 30, 'learning_rate': 1.4527195, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:10,289] Trial 106 finished with value: 0.6967467079783114 and parameters: {'n_estimators': 36, 'learning_rate': 2.2768295, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:11,027] Trial 107 finished with value: 0.809004329004329 and parameters: {'n_estimators': 18, 'learning_rate': 2.4738405, 'random_s

[I 2023-12-15 00:05:50,891] Trial 142 finished with value: 0.4044692737430167 and parameters: {'n_estimators': 24, 'learning_rate': 0.6807654999999999, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:51,863] Trial 143 finished with value: 0.809004329004329 and parameters: {'n_estimators': 24, 'learning_rate': 2.4583875, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:52,991] Trial 144 finished with value: 0.809004329004329 and parameters: {'n_estimators': 28, 'learning_rate': 2.4079775, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:53,881] Trial 145 finished with value: 0.809004329004329 and parameters: {'n_estimators': 22, 'learning_rate': 2.3598905, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:05:54,542] Trial 146 finished with value: 0.809004329004329 and parameters: {'n_estimators': 16, 'learning_rate': 2.290329, '

[I 2023-12-15 00:06:37,228] Trial 181 finished with value: 0.809004329004329 and parameters: {'n_estimators': 24, 'learning_rate': 2.100539, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:06:37,653] Trial 182 finished with value: 0.809004329004329 and parameters: {'n_estimators': 10, 'learning_rate': 2.2431325, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:06:38,472] Trial 183 finished with value: 0.809004329004329 and parameters: {'n_estimators': 20, 'learning_rate': 2.4254145, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:06:39,516] Trial 184 finished with value: 0.809004329004329 and parameters: {'n_estimators': 26, 'learning_rate': 2.333979, 'random_state': 2023}. Best is trial 4 with value: 0.809004329004329.
[I 2023-12-15 00:06:41,508] Trial 185 finished with value: 0.19553502694380293 and parameters: {'n_estimators': 50, 'learning_rate': 2.189, 'random_state'

In [17]:
pd.DataFrame(y_valid).value_counts()

label
1        2336
0        1103
dtype: int64

In [17]:
pd.DataFrame(total_list1,columns = ['time_lag','model','estimator','acc','f1_score'])

,time_lag,model,estimator,acc,f1_score
0,0,xgb,"{'n_estimators': 816, 'max_depth': 19, 'learni...",0.603482,0.530075
1,0,lgbm,"{'num_leaves': 418, 'n_estimators': 70, 'max_d...",0.785927,0.750145
2,1,xgb,"{'n_estimators': 566, 'max_depth': 15, 'learni...",0.645880,0.540596
3,1,lgbm,"{'num_leaves': 153, 'n_estimators': 34, 'max_d...",0.839639,0.808119
4,2,xgb,"{'n_estimators': 452, 'max_depth': 3, 'learnin...",0.793500,0.746876
5,2,lgbm,"{'num_leaves': 117, 'n_estimators': 76, 'max_d...",0.860578,0.824760
6,3,xgb,"{'n_estimators': 564, 'max_depth': 5, 'learnin...",0.600347,0.561110
7,3,lgbm,"{'num_leaves': 382, 'n_estimators': 7, 'max_de...",0.790991,0.750650
8,4,xgb,"{'n_estimators': 911, 'max_depth': 4, 'learnin...",0.642781,0.466609
9,4,lgbm,"{'num_leaves': 500, 'n_estimators': 63, 'max_d...",0.697802,0.651698


In [10]:
total_list1

[[0,
  'cat',
  {'iterations': 26,
   'od_wait': 301,
   'reg_lambda': 91.9164297936471,
   'bagging_temperature': 0.03282775426654037,
   'learning_rate': 0.2768818655458132,
   'leaf_estimation_iterations': 8,
   'depth': 4,
   'min_data_in_leaf': 12,
   'random_state': 2023,
   'boosting_type': 'Plain'},
  0.6978494623655914,
  0.6749566223250434],
 [1,
  'cat',
  {'iterations': 28,
   'od_wait': 305,
   'reg_lambda': 75.57773189920198,
   'bagging_temperature': 0.05875324626145144,
   'learning_rate': 0.24151378977476057,
   'leaf_estimation_iterations': 5,
   'depth': 4,
   'min_data_in_leaf': 3,
   'random_state': 2023,
   'boosting_type': 'Plain'},
  0.6179302045728039,
  0.6290887850467289],
 [2,
  'cat',
  {'iterations': 27,
   'od_wait': 377,
   'reg_lambda': 84.32089262047836,
   'bagging_temperature': 2.356149703515959,
   'learning_rate': 0.2952783548580561,
   'leaf_estimation_iterations': 10,
   'depth': 3,
   'min_data_in_leaf': 12,
   'random_state': 2023,
   'boosting